In [1]:
import os
import sklearn


In [2]:
from loguru import logger
logger.info(sklearn.__version__)


2024-06-11 19:52:48.740 | INFO     | __main__:<module>:2 - 1.5.0


In [3]:
import sys
logger.info(sys.version)


2024-06-11 19:52:48.751 | INFO     | __main__:<module>:2 - 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:14:58) [MSC v.1929 64 bit (AMD64)]


In [4]:
import pickle
import pandas as pd


In [5]:
## variables
year = 2023
month = 3
taxi_type = "yellow"

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)
    

In [7]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df


In [8]:
df = read_data(f'./data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet')


In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)


## Q1. Notebook

We'll start with the same notebook we ended up with in homework 1. We cleaned it a little bit and kept only the scoring part. You can find the initial notebook here.

Run this notebook for the March 2023 data.

What's the standard deviation of the predicted duration for this dataset?


In [10]:
y_pred.std()

1.3918725592180876

## Q2. Preparing the output

Like in the course videos, we want to prepare the dataframe with the output. 

First, let's create an artificial `ride_id` column:

```python
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results. 

Save it as parquet:

```python
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```

What's the size of the output file?


In [11]:
df_result = pd.DataFrame()
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


In [12]:
df_result['predict'] = y_pred


In [13]:
df_result.head(20)

,ride_id,predict
0,2023/03_0,0.871898
1,2023/03_1,1.454797
2,2023/03_2,-1.205457
3,2023/03_3,-0.019220
4,2023/03_4,0.080646
5,2023/03_5,0.953962
6,2023/03_6,1.336841
7,2023/03_7,1.185406
8,2023/03_8,2.166346
9,2023/03_9,0.816866


In [14]:
output_folder = os.path.join('output', taxi_type)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
output_file = os.path.join(output_folder, f'result_{year:04d}-{month:02d}.parquet')


In [15]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

## Q3. Creating the scoring script

Now let's turn the notebook into a script. 

Which command you need to execute for that?


## Q4. Virtual environment

Now let's put everything into a virtual environment. We'll use pipenv for that.

Install all the required libraries. Pay attention to the Scikit-Learn version: it should be the same as in the starter
notebook.

After installing the libraries, pipenv creates two files: `Pipfile`
and `Pipfile.lock`. The `Pipfile.lock` file keeps the hashes of the
dependencies we use for the virtual env.

What's the first hash for the Scikit-Learn dependency?


## Q5. Parametrize the script

Let's now make the script configurable via CLI. We'll create two 
parameters: year and month.

Run the script for April 2023. 

What's the mean predicted duration? 

* 7.29
* 14.29
* 21.29
* 28.29

Hint: just add a print statement to your script.

## Q6. Docker container 

Finally, we'll package the script in the docker container. 
For that, you'll need to use a base image that we prepared. 

This is what the content of this image is:
```
FROM python:3.10.13-slim

WORKDIR /app
COPY [ "model2.bin", "model.bin" ]
```

Note: you don't need to run it. We have already done it.

It is pushed it to [`agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim`](https://hub.docker.com/layers/agrigorev/zoomcamp-model/mlops-2024-3.10.13-slim/images/sha256-f54535b73a8c3ef91967d5588de57d4e251b22addcbbfb6e71304a91c1c7027f?context=repo),
which you need to use as your base image.

That is, your Dockerfile should start with:

```docker
FROM agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

# do stuff here
```

This image already has a pickle file with a dictionary vectorizer
and a model. You will need to use them.

Important: don't copy the model to the docker image. You will need
to use the pickle file already in the image. 

Now run the script with docker. What's the mean predicted duration
for May 2023? 

* 0.19
* 7.24
* 14.24
* 21.19
